## background 
- 32% survival
- women,children,upper class more likely survived
- not enough boats

## load utils

In [1]:
import sys
sys.path.append('../')
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Imputer
from util import *
from onehot import LabelBinarizerEx
from pipeline import FeaturePipeline, DataFramePipeline
from binning import Binner
from title import TitleExtractor
from cabin import HasCabin
from ensemble import EnsembleStackClassifierEx
from addcols import AddColumns
from impute import GroupImputer,MixImputer
from alone import IsAlone
from scipy.stats import boxcox
import pandas as pd



d:\app\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## load train data

In [2]:
passenger_train=pd.read_csv('raw_data/train.csv')
passenger_test=pd.read_csv('raw_data/test.csv')
target_col='Survived'
id_col='PassengerId'
target=passenger_train[target_col]
total_num=len(passenger_train)

In [3]:
passenger_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
passenger_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
passenger_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [6]:
passenger_train.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Hirvonen, Miss. Hildur E",male,347082,G6,S
freq,1,577,7,4,644


In [7]:
passenger_test.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,418,418,418,91,418
unique,418,2,363,76,3
top,"Elias, Mr. Joseph",male,PC 17608,B57 B59 B63 B66,S
freq,1,266,5,3,270


In [8]:
raw_df=pd.concat([passenger_train.drop(target_col,axis=1),passenger_test]).drop(id_col,axis=1)
raw_df.index=np.arange(len(raw_df))
org_cols=raw_df.columns

age_group=raw_df.groupby(['Pclass','Sex']).mean()['Age']
missing_index=raw_df[raw_df.Age.isnull()].index
for i in missing_index:
    raw_df.loc[i,'Age'] = age_group[(raw_df.loc[i,'Pclass'],raw_df.loc[i,'Sex'])]
    
raw_df.Fare=Imputer(strategy='mean').fit_transform(raw_df.Fare.values.reshape(-1,1))

raw_df['FamilySize']=raw_df.SibSp+raw_df.Parch

raw_df['IsAlone']=(raw_df.FamilySize==0).astype('int')

def countCabin(cabin):
    c=0
    if cabin is not np.nan:
        c=len(cabin.split())
    return c

raw_df['Cabins']=raw_df.Cabin.apply(countCabin)

for c in ['Pclass','Age','Fare','SibSp','Parch','FamilySize','Cabins']:
    new_c=c+'_Norm'
    raw_df[new_c]=raw_df[c]
    if raw_df[new_c].min()<=0.:
        raw_df[new_c]=raw_df[new_c]+abs(raw_df[new_c].min())+0.1
    tranformed,_=boxcox(raw_df[new_c])
    raw_df[new_c]=StandardScaler().fit_transform(tranformed.reshape(-1,1))
    raw_df.drop(c,axis=1,inplace=True)
import re

# def extractTitle(name):
#     m = re.search(' \w+\\.',name)
#     if m:
#         return m.group()[1:-1]
#     else:
#         return np.nan
    
def extractTitle(name):
    title=np.nan
    m = re.search(' \w+\\.',name)
    if m:
        t=m.group()[1:-1]
        if t in ['Mr','Miss','Mrs','Master']:
            title = t
    return title
    
raw_df['Title']=raw_df.Name.apply(extractTitle)

# def extractTicketNumber(ticket):
#     try:
#         return float(ticket)
#     except:
#         splits=ticket.split()
#         if len(splits)>1:
#             return float(splits[1])
#         else:
#             return np.nan
        
# raw_df['Ticket_Number']=raw_df.Ticket.apply(extractTicketNumber)

# def extractTicketLocation(ticket):
#     m = re.search('\w+ ',ticket)
#     if m:
#         return m.group()[0]
#     else:
#         return np.nan

# raw_df['Ticket_Location']=raw_df.Ticket.apply(extractTicketLocation)

def isTicketNum(ticket):
    try:
        int(ticket)
        return 1
    except Exception as e:
        return 0
    
# raw_df['IsTicketNum']=raw_df.Ticket.apply(isTicketNum)

raw_df['HasCabin']=(raw_df.Cabin.isnull()==False).astype('int')

raw_df['Embarked_Imp']=MixImputer().fit_transform(raw_df[['Embarked']])

raw_df=pd.get_dummies(raw_df,columns=['Sex','Title','Embarked_Imp'])

pre_df=raw_df.drop(['Name','Ticket','Cabin','Embarked'],axis=1)
pre_df.columns

Index(['IsAlone', 'Pclass_Norm', 'Age_Norm', 'Fare_Norm', 'SibSp_Norm',
       'Parch_Norm', 'FamilySize_Norm', 'Cabins_Norm', 'HasCabin',
       'Sex_female', 'Sex_male', 'Title_Master', 'Title_Miss', 'Title_Mr',
       'Title_Mrs', 'Embarked_Imp_C', 'Embarked_Imp_Q', 'Embarked_Imp_S'],
      dtype='object')

In [9]:
train_data=pre_df[:len(passenger_train)]
test_data=pre_df[len(passenger_train):]
len(train_data)==len(passenger_train)

True

In [10]:
train_data.describe()

,IsAlone,Pclass_Norm,Age_Norm,Fare_Norm,SibSp_Norm,Parch_Norm,FamilySize_Norm,Cabins_Norm,HasCabin,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Embarked_Imp_C,Embarked_Imp_Q,Embarked_Imp_S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.602694,0.017178,-0.007725,-0.021887,-0.002092,0.010635,0.002969,0.008422,0.228956,0.352413,0.647587,0.044893,0.204265,0.580247,0.140292,0.188552,0.086420,0.725028
std,0.489615,0.998623,1.013282,1.013225,1.001519,1.007094,1.002780,1.005835,0.420397,0.477990,0.477990,0.207186,0.403390,0.493796,0.347485,0.391372,0.281141,0.446751
min,0.000000,-1.472855,-2.699889,-3.958510,-0.684350,-0.553478,-0.805476,-0.539368,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,-0.509118,-0.509589,-0.753117,-0.684350,-0.553478,-0.805476,-0.539368,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.869898,-0.196732,-0.230710,-0.684350,-0.553478,-0.805476,-0.539368,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.869898,0.610768,0.467671,1.419425,-0.553478,1.052411,-0.539368,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,0.869898,3.350701,3.410959,1.657046,1.856903,1.624263,1.891378,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
tmp=train_data.copy()
tmp['Survived']=target
tmp.corr()['Survived'].sort_values(ascending=False)

Survived           1.000000
Sex_female         0.543351
Title_Mrs          0.339040
Title_Miss         0.327093
Fare_Norm          0.320527
HasCabin           0.316912
Cabins_Norm        0.316763
FamilySize_Norm    0.187595
Embarked_Imp_C     0.168240
Parch_Norm         0.146692
SibSp_Norm         0.108983
Title_Master       0.085221
Embarked_Imp_Q     0.003650
Age_Norm          -0.073732
Embarked_Imp_S    -0.149683
IsAlone           -0.203367
Pclass_Norm       -0.339811
Sex_male          -0.543351
Title_Mr          -0.549199
Name: Survived, dtype: float64

## train

In [22]:
last_result={}
from imp import reload
from aml import auto_model_machine as aml

In [38]:
TASK_NAME='stacking-best2-lr'
logger,handlers=initLogging(TASK_NAME)

aml=reload(aml)
bc=aml.BinaryClassifier.auto(train_data,target,('lr','svc','rf','gbt','mlp','ada','ext'),last_result,logger=logger,cv=5,best_n=1)
resetLogging(logger,handlers)
bc.fit(train_data.values,target)



2017-11-18 12:05:17,376 - begin [0] for LogisticRegression {C = 0.001, penalty = l1}
2017-11-18 12:05:17,376 - score 0.616 time 0
2017-11-18 12:05:17,376 - begin [1] for LogisticRegression {C = 0.001, penalty = l2}
2017-11-18 12:05:17,393 - score 0.751 time 0
2017-11-18 12:05:17,394 - begin [2] for LogisticRegression {C = 0.01, penalty = l1}
2017-11-18 12:05:17,396 - score 0.789 time 0
2017-11-18 12:05:17,398 - begin [3] for LogisticRegression {C = 0.01, penalty = l2}
2017-11-18 12:05:17,400 - score 0.789 time 0
2017-11-18 12:05:17,402 - begin [4] for LogisticRegression {C = 0.1, penalty = l1}
2017-11-18 12:05:17,406 - score 0.796 time 0
2017-11-18 12:05:17,408 - begin [5] for LogisticRegression {C = 0.1, penalty = l2}
2017-11-18 12:05:17,410 - score 0.814 time 0
2017-11-18 12:05:17,412 - begin [6] for LogisticRegression {C = 1, penalty = l1}
2017-11-18 12:05:17,415 - score 0.823 time 0
2017-11-18 12:05:17,417 - begin [7] for LogisticRegression {C = 1, penalty = l2}
2017-11-18 12:05:17

2017-11-18 12:05:18,351 - score 0.820 time 0
2017-11-18 12:05:18,353 - begin [53] for RandomForestClassifier {max_depth = 15, max_features = sqrt, n_estimators = 500}
2017-11-18 12:05:18,375 - score 0.818 time 0
2017-11-18 12:05:18,377 - begin [54] for GradientBoostingClassifier {learning_rate = 0.01, max_depth = 3, max_features = None, n_estimators = 100, subsample = 1.0}
2017-11-18 12:05:18,405 - score 0.818 time 0
2017-11-18 12:05:18,407 - begin [55] for GradientBoostingClassifier {learning_rate = 0.01, max_depth = 3, max_features = None, n_estimators = 100, subsample = 0.8}
2017-11-18 12:05:18,433 - score 0.820 time 0
2017-11-18 12:05:18,436 - begin [56] for GradientBoostingClassifier {learning_rate = 0.01, max_depth = 3, max_features = None, n_estimators = 100, subsample = 0.6}
2017-11-18 12:05:18,469 - score 0.811 time 0
2017-11-18 12:05:18,471 - begin [57] for GradientBoostingClassifier {learning_rate = 0.01, max_depth = 3, max_features = None, n_estimators = 500, subsample = 1.

2017-11-18 12:05:19,672 - score 0.831 time 0
2017-11-18 12:05:19,675 - begin [93] for GradientBoostingClassifier {learning_rate = 0.01, max_depth = 8, max_features = None, n_estimators = 500, subsample = 1.0}
2017-11-18 12:05:19,705 - score 0.814 time 0
2017-11-18 12:05:19,708 - begin [94] for GradientBoostingClassifier {learning_rate = 0.01, max_depth = 8, max_features = None, n_estimators = 500, subsample = 0.8}
2017-11-18 12:05:19,742 - score 0.817 time 0
2017-11-18 12:05:19,743 - begin [95] for GradientBoostingClassifier {learning_rate = 0.01, max_depth = 8, max_features = None, n_estimators = 500, subsample = 0.6}
2017-11-18 12:05:19,772 - score 0.818 time 0
2017-11-18 12:05:19,774 - begin [96] for GradientBoostingClassifier {learning_rate = 0.01, max_depth = 8, max_features = None, n_estimators = 1000, subsample = 1.0}
2017-11-18 12:05:19,804 - score 0.809 time 0
2017-11-18 12:05:19,809 - begin [97] for GradientBoostingClassifier {learning_rate = 0.01, max_depth = 8, max_features

2017-11-18 12:05:21,083 - begin [132] for GradientBoostingClassifier {learning_rate = 0.1, max_depth = 5, max_features = None, n_estimators = 1000, subsample = 1.0}
2017-11-18 12:05:21,117 - score 0.797 time 0
2017-11-18 12:05:21,119 - begin [133] for GradientBoostingClassifier {learning_rate = 0.1, max_depth = 5, max_features = None, n_estimators = 1000, subsample = 0.8}
2017-11-18 12:05:21,160 - score 0.819 time 0
2017-11-18 12:05:21,162 - begin [134] for GradientBoostingClassifier {learning_rate = 0.1, max_depth = 5, max_features = None, n_estimators = 1000, subsample = 0.6}
2017-11-18 12:05:21,199 - score 0.818 time 0
2017-11-18 12:05:21,201 - begin [135] for GradientBoostingClassifier {learning_rate = 0.1, max_depth = 5, max_features = sqrt, n_estimators = 100, subsample = 1.0}
2017-11-18 12:05:21,240 - score 0.832 time 0
2017-11-18 12:05:21,243 - begin [136] for GradientBoostingClassifier {learning_rate = 0.1, max_depth = 5, max_features = sqrt, n_estimators = 100, subsample = 0.

2017-11-18 12:05:23,129 - score 0.762 time 0
2017-11-18 12:05:23,131 - begin [176] for SVC {C = 1.0, coef0 = 1.0, gamma = 0.001, kernel = poly}
2017-11-18 12:05:23,185 - score 0.776 time 0
2017-11-18 12:05:23,187 - begin [177] for SVC {C = 1.0, coef0 = 1.0, gamma = 0.01, kernel = linear}
2017-11-18 12:05:23,243 - score 0.791 time 0
2017-11-18 12:05:23,244 - begin [178] for SVC {C = 1.0, coef0 = 1.0, gamma = 0.01, kernel = rbf}
2017-11-18 12:05:23,285 - score 0.793 time 0
2017-11-18 12:05:23,288 - begin [179] for SVC {C = 1.0, coef0 = 1.0, gamma = 0.01, kernel = poly}
2017-11-18 12:05:23,331 - score 0.797 time 0
2017-11-18 12:05:23,333 - begin [180] for SVC {C = 1.0, coef0 = 1.0, gamma = 0.1, kernel = linear}
2017-11-18 12:05:23,375 - score 0.791 time 0
2017-11-18 12:05:23,379 - begin [181] for SVC {C = 1.0, coef0 = 1.0, gamma = 0.1, kernel = rbf}
2017-11-18 12:05:23,423 - score 0.814 time 0
2017-11-18 12:05:23,426 - begin [182] for SVC {C = 1.0, coef0 = 1.0, gamma = 0.1, kernel = poly}

2017-11-18 12:05:26,211 - score 0.819 time 0
2017-11-18 12:05:26,213 - begin [233] for SVC {C = 10.0, coef0 = 10.0, gamma = auto, kernel = poly}
2017-11-18 12:05:26,266 - score 0.814 time 0
2017-11-18 12:05:26,267 - begin [234] for MLPClassifier {activation = relu, hidden_layer_sizes = (50,)}
2017-11-18 12:05:26,272 - score 0.827 time 0
2017-11-18 12:05:26,275 - begin [235] for MLPClassifier {activation = relu, hidden_layer_sizes = (100,)}
2017-11-18 12:05:26,278 - score 0.833 time 0
2017-11-18 12:05:26,281 - begin [236] for MLPClassifier {activation = relu, hidden_layer_sizes = (200,)}
2017-11-18 12:05:26,286 - score 0.836 time 0
2017-11-18 12:05:26,290 - begin [237] for MLPClassifier {activation = identity, hidden_layer_sizes = (50,)}
2017-11-18 12:05:26,295 - score 0.816 time 0
2017-11-18 12:05:26,297 - begin [238] for MLPClassifier {activation = identity, hidden_layer_sizes = (100,)}
2017-11-18 12:05:26,301 - score 0.815 time 0
2017-11-18 12:05:26,307 - begin [239] for MLPClassifie

2017-11-18 12:05:26,891 - begin [285] for ExtraTreesClassifier {max_depth = 8, max_features = None, n_estimators = 500}
2017-11-18 12:05:26,904 - score 0.821 time 0
2017-11-18 12:05:26,906 - begin [286] for ExtraTreesClassifier {max_depth = 8, max_features = sqrt, n_estimators = 10}
2017-11-18 12:05:26,917 - score 0.818 time 0
2017-11-18 12:05:26,922 - begin [287] for ExtraTreesClassifier {max_depth = 8, max_features = sqrt, n_estimators = 50}
2017-11-18 12:05:26,940 - score 0.817 time 0
2017-11-18 12:05:26,942 - begin [288] for ExtraTreesClassifier {max_depth = 8, max_features = sqrt, n_estimators = 100}
2017-11-18 12:05:26,953 - score 0.817 time 0
2017-11-18 12:05:26,955 - begin [289] for ExtraTreesClassifier {max_depth = 8, max_features = sqrt, n_estimators = 500}
2017-11-18 12:05:26,971 - score 0.817 time 0
2017-11-18 12:05:26,977 - begin [290] for ExtraTreesClassifier {max_depth = 8, max_features = log2, n_estimators = 10}
2017-11-18 12:05:26,990 - score 0.818 time 0
2017-11-18 12

In [25]:
resetLogging(logger,handlers)

In [39]:
from sklearn.linear_model import LogisticRegression,SGDClassifier,LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

base_clfs=[
    bc,
#     bc.get_nth_model(0),
#     bc.get_nth_model(1),
#     SVC(probability=True,random_state=42,C=10.0,gamma=0.01,coef0=1.,degree=3,kernel='poly'),
#     GradientBoostingClassifier(learning_rate=0.01,max_depth=5,max_features=None, n_estimators=100)
    LogisticRegression(random_state=42),
    SVC(probability=True,random_state=42),
    RandomForestClassifier(random_state=42),
    GradientBoostingClassifier(random_state=42),
    AdaBoostClassifier(), 
    ExtraTreesClassifier(n_jobs=-1), 
#       XGBClassifier(),
]
from sklearn.model_selection import cross_val_score

%matplotlib inline
import matplotlib.pyplot as plt
from pylab import *

def trainModels(train_data, target):
    results=[]
    for clf in base_clfs:
        scores = cross_val_score(clf,train_data,target,scoring='accuracy',cv=5)
        results.append((clf.__class__.__name__[:3],np.mean(scores),np.std(scores)))
    for r in sorted(results,key=lambda x:x[1],reverse=True):
        print('%s %.3f %.3f'%r)
#     labels=[c.__class__.__name__[:3] for c in base_clfs]
#     X=np.arange(len(base_clfs))
#     bar(X,scores,tick_label=labels,color='rgb')
#     show()
#     print(sorted(zip(labels,scores),key=lambda x:x[1],reverse=True))
    
trainModels(train_data.values,target.values)

d:\app\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
d:\app\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
d:\app\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
d:\app\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
d:\app\Anaconda3\lib

Gra 0.836 0.017
SVC 0.825 0.024
Bin 0.824 0.032
Log 0.816 0.017
Ran 0.811 0.035
Ada 0.809 0.014
Ext 0.806 0.043


In [16]:
# bc.get_nth_model(0)
# base_clfs[2]
GradientBoostingClassifier()

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [42]:
TASK_NAME='gbt-01-3-None-500-08'
clf=GradientBoostingClassifier(learning_rate=0.1,max_depth=3,max_features=None,n_estimators=500,subsample=0.8)
# clf.fit(train_data,target)
# bc.select(0)
# clf=bc
# clf=base_clfs[4]
# clf=bc.get_nth_model(0)
# clf=GradientBoostingClassifier(learning_rate=0.1,max_depth=3,max_features=None,n_estimators=500,subsample=0.8)
# clf=GradientBoostingClassifier(n_estimators=500)
clf.fit(train_data.values,target)
# bc.fit_one(2,train_data.values,target)
test_id=passenger_test[id_col]
# clf=base_clfs[0]
survived=clf.predict(test_data)
test_Survived = pd.Series(survived, name="Survived").astype(int)
results = pd.concat([test_id,test_Survived],axis=1)
results.to_csv("output/%s.csv"%TASK_NAME,index=False)